# Import libraries.

In [ ]:
import torch
import pandas
import numpy as np
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold
from statistics import mode
import statistics

# Load the dataset.

The following step is just to test that csv file is loading correctly.

In [ ]:
dataset_torch = pandas.read_csv('sao-paulo-properties-april-2019.csv', usecols = ["Rooms", "Size", "Toilets", "Parking", "Price"])
dataset_torch.head(4)

,Price,Size,Rooms,Toilets,Parking
0,10000000,343,4,7,5
1,9979947,343,4,6,5
2,8500000,420,4,6,4
3,8039200,278,4,7,4


## Preprocessing the data.

The dataset can be found in the following url: https://www.kaggle.com/yashsawarn/wifi-stretgth-for-rooms. We are using for this example a couple of atributes from this file, such as: ``Rooms, Size, Toilets and Parking``.

In [ ]:

# read_dataset: 
# Read a csv file and returns a pytorch tensor.
def read_dataset(csv_name = 'sao-paulo-properties-april-2019.csv'):

    # Filter with columns are we gonna used from the original csv dataset.
    dataset_torch = pandas.read_csv(csv_name, usecols = ["Rooms", "Size", "Toilets", "Parking", "Price"])
    # Create a new column with the results of apply 3 conditions to the Price value.
    conditions = [
      (dataset_torch["Price"] > 900000 ),
      (dataset_torch["Price"] > 580000) & (dataset_torch["Price"] <= 900000),
      (dataset_torch["Price"] > 400000) & (dataset_torch["Price"] <= 580000),
      (dataset_torch["Price"] <= 400000)
    ]
    # Label for eache category described before.
    values = [4,3,2,1]
    dataset_torch["Category"] = np.select(conditions, values)
    dataset_torch = dataset_torch.drop(columns = ["Price"])
    
    # Return the generated dataframe in torch format.
    return torch.from_numpy(dataset_torch.values)

In this step we load the data doing a call to the `read_dataset` function.

In [ ]:
dataset_torch = read_dataset()

In [ ]:
categorias, total_obs = dataset_torch[:,4].unique(dim = 0, return_counts = True)
print(categorias)
print(total_obs)

tensor([1, 2, 3, 4])
tensor([1913,  998,  998,  985])


# Create the class Node_CART to genere each node to build a tree.

In [ ]:
class Node_CART:    
    def __init__(self, num_classes = 4, ref_CART = None, current_depth = 0):
        """
        Create the node attributes  
        param num_classes: K number of classes to classify param ref_cart: reference to the tree containing the node
        param current_depth: current depth of the node in the tree
        """
        self.ref_CART = ref_CART
        self.threshold_value = 0
        self.feature_num = 0
        self.node_right = None
        self.node_left = None
        self.data_torch_partition = None
        self.gini = 0
        self.dominant_class = None
        self.accuracy_dominant_class = None        
        self.num_classes = num_classes
        self.current_depth = current_depth
    
    #  Recursive function to write the node content to an xml formatted string param current_str : the xml content so far in the whole tree return the string with the node content
    def to_xml(self, current_str = ""):
        str_node = "<node><thresh>" + str(self.threshold_value) + "</thresh>" + "<feature>" + str(self.feature_num) + "</feature><depth>" + str(self.current_depth)+ "</depth>" 
        str_node += "<gini>" + str(self.gini) + "</gini>"
        if(self.node_right != None):
            str_left = self.node_right.to_xml(current_str)
            str_node += str_left
        if(self.node_left != None):
            str_right = self.node_left.to_xml(current_str)
            str_node += str_right
            
        if(self.is_leaf()):
            str_node += "<dominant_class>" + str(self.dominant_class) + "</dominant_class><acc_dominant_class>"  + str(self.accuracy_dominant_class) + "</acc_dominant_class>"
        str_node += "</node>"
        return str_node
    
    # Checks whether the node is a leaf
    def is_leaf(self):
        return (self.node_left == None and self.node_right == None)
    
    
    def create_with_children(self, data_torch, current_depth, list_selected_features = [], min_gini = 0.000001):
        """
        Creates a node by selecting the best feature and threshold, and if needed, creating its children
        param data_torch: dataset with the current partition to deal with in the node
        param current_depth: depth counter for the node
        param list_selected_features: list of selected features so far for the CART building process
        param min_gini: hyperparmeter selected by the user defining the minimum tolerated gini coefficient for a  node
        return the list of selected features so far
        """        
        #update depth of children
        depth_children = current_depth + 1
        if(depth_children <= self.ref_CART.get_max_depth()):

            num_observations = data_torch.shape[0]            
            # careful with max depth
            #if no threshold and feature were selected, select it using a greedy approach            
            (threshold_value, feature_num, gini) = self.select_best_feature_and_thresh(data_torch, list_features_selected = list_selected_features)
            list_selected_features += [feature_num]
            #store important data in attributes
            self.threshold_value = threshold_value
            self.feature_num = feature_num
            self.data_torch_partition = data_torch
            self.gini = gini            
            num_features = data_torch.shape[1]
            #data_torch_left = torch.zeros(1, num_features)
            #data_torch_right = torch.zeros(1, num_features)
            #create the right and left node data if the current gini is still high            
            if(self.gini > min_gini):                
                data_torch_left = data_torch[data_torch[:, feature_num] < threshold_value]
                data_torch_right = data_torch[data_torch[:, feature_num] >= threshold_value]
                #if the new partitions have more than min_observations, make them
                if(data_torch_left.shape[0] >= self.ref_CART.get_min_observations() and data_torch_right.shape[0] >= self.ref_CART.get_min_observations()):
                    #add data to the right and left children
                    self.node_right = Node_CART(num_classes = self.num_classes, ref_CART = self.ref_CART, current_depth = depth_children)
                    self.node_left = Node_CART(num_classes = self.num_classes, ref_CART = self.ref_CART, current_depth = depth_children)
                    list_selected_features = self.node_right.create_with_children(data_torch_right, depth_children, list_selected_features = list_selected_features)            
                    self.node_left.create_with_children( data_torch_left, depth_children, list_selected_features = list_selected_features)
        #if is leaf, fill the         
        if(self.is_leaf()):            
            labels_data = data_torch[:,  -1]
            self.dominant_class = torch.mode(labels_data).values.item()
            num_obs_label = labels_data[labels_data == self.dominant_class].shape[0]
            self.accuracy_dominant_class = num_obs_label / labels_data.shape[0]           
            
        return list_selected_features


    def calculate_gini(self, data_partition_torch, num_classes = 4):
        """
        Calculates the gini coefficient for a given partition with the given number of classes
        param data_partition_torch: current dataset partition as a tensor
        param num_classes: K number of classes to discriminate from
        returns the calculated gini coefficient
        """
        #TODO
        n = data_partition_torch.size()[0] 
        if(n != 0):
            categorias, total_obs = data_partition_torch[:,4].unique(dim = 0, return_counts = True)
            suma = sum((total_obs/n)**2)
            gini = 1 - suma
        else:
            gini = 1
        return gini
      
    def calculate_rss(self, data_partition_torch):
        """
        Calculates the Residual Sum of Squares (RSS)
        param data_partition_torch: current dataset partition as a tensor
        returns the calculated RSS
        """
        rss = (data_partition_torch - torch.mean(data_partition_torch))**2
        rss = torch.sum(rss)
        return rss


         
    def select_best_feature_and_thresh(self, data_torch, list_features_selected = [], num_classes = 4):
        """
        ONLY USE  2 FORS
        Selects the best feature and threshold that minimizes the gini coefficient
        param data_torch: dataset partition to analyze
        param list_features_selected list of features selected so far, thus must be ignored 
        param num_classes: number of K classes to discriminate from 
        return min_thresh, min_feature, min_gini found for the dataset partition when 
        selecting the found feature and threshold
        """     
          
        n,n_i,n_d = 0,0,0
        min_thresh = -1
        min_feature = -1
        min_gini = 10

        for i in range(data_torch.size()[1]-1):
          data_feature = data_torch[:, i].unsqueeze(1)
          unique_values = torch.unique(data_torch[:,i], sorted  = True)
          n = data_torch.size()[0]

          for j in unique_values:
              D_izq = data_torch[data_torch[:,i] < j.item()]
              D_der = data_torch[data_torch[:,i] >= j.item()]
              n_i = D_izq.size()[0]
              n_d = D_der.size()[0]
              #rss = self.calculate_rss(D_izq) + self.calculate_rss(D_der)
              coef_gini = (n_i/n)*self.calculate_gini(D_izq) + (n_d/n)*self.calculate_gini(D_der)
              if(coef_gini < min_gini):
                  min_gini = coef_gini
                  min_feature = i
                  min_thresh = j.item()        
        # print(min_thresh, min_feature, min_gini)     
        return (min_thresh, min_feature, min_gini)  
           
    
    def evaluate_node(self, input_torch): 
        """
        Evaluates an input observation within the node. 
        If is not a leaf node, send it to the corresponding node
        return predicted label
        """
        feature_val_input = input_torch[self.feature_num]
        if(self.is_leaf()):
            return self.dominant_class
        else:
            if(feature_val_input < self.threshold_value):
                return self.node_left.evaluate_node(input_torch)
            else:
                return self.node_right.evaluate_node(input_torch)

# Create the class CART who contains the root node of the tree.

In [ ]:
# This class only contents the root node.
class CART:
    def __init__(self, dataset_torch, max_CART_depth = 4, min_observations = 2):
        # min observations per node
        self.min_observations = min_observations
        # Create a new root node to create the tree.
        self.root = Node_CART(num_classes = 4, ref_CART = self, current_depth = 0)
        self.max_CART_depth = max_CART_depth
        self.list_selected_features = []
    
    # Gets the root node of the tree.
    def get_root(self):     
        return self.root
    
    # Returns the min of observations per node.
    def get_min_observations(self): 
        return self.min_observations
    
    # Get the max depth of the tree.
    def get_max_depth(self):    
        return self.max_CART_depth
    
    # Buiild a CART tree from root.
    def build_CART(self, data_torch):
        self.list_selected_features = self.root.create_with_children(data_torch, current_depth = 0)
    
    # Write a XML file with the tree content.
    def to_xml(self, xml_file_name): 
        str_nodes = self.root.to_xml()
        file = open(xml_file_name,"w+") 
        file.write(str_nodes)
        file.close()
        return str_nodes
    
    # Evaluate a specifc input in the tree and get the predicted class.
    def evaluate_input(self, input_torch):
        """
        Evaluate a specific input in the tree and get the predicted class
        """
        return self.root.evaluate_node(input_torch)

# Create the class Train_CART who create a whole tree structure.

In [ ]:
# Use to generate a whole tree structure of a CART model.    
def train_CART(dataset_torch, name_xml = "", max_CART_depth = 3, min_obs_per_leaf = 2): 
    # Create an element from CART class.
    tree = CART(dataset_torch = dataset_torch, max_CART_depth = max_CART_depth, min_observations =  min_obs_per_leaf)
    # Generate the children for the tree.
    tree.build_CART(dataset_torch)
    # Call to generate a XML file with tree content.
    if(not name_xml == ""):
        tree.to_xml(name_xml)

    # Return the tree created.
    return tree

# Evaluate a CART tree.

In [ ]:
# Test a previously built CART tree.
def test_CART(tree, testset_torch):
    sum  = 0
    for i in range(testset_torch.size()[0]):
        result = tree.evaluate_input(testset_torch[i])
        if testset_torch[i][4] == result:
            sum +=1
    accuracy = sum / testset_torch.size()[0] * 100;
    return accuracy

# Partition Validation

In [ ]:
def partition_validation(dataset_torch, max_CART_depth, num_splits):
    rs = ShuffleSplit(n_splits = num_splits, test_size=.30)
    cont_part = 1
    results = []
    for train_index, test_index in rs.split(dataset_torch):
        print("Evaluación de la partion %s:" % (cont_part))
        train = dataset_torch[train_index]
        test = dataset_torch[test_index]
        # Train a new tree for a specific dataset.
        tree = train_CART(train, max_CART_depth = max_CART_depth, name_xml = "CART_example.xml")
        # Test the CART model.
        acc = test_CART(tree, test)
        print("Acc: %s" % (acc))
        cont_part +=1  
        results.append(acc)
    return(results)    
        

# Evaluation CART

### Evaluation CART with depth 2.

In [ ]:
# Train a new tree for a specific dataset.
tree = train_CART(dataset_torch, max_CART_depth = 2, name_xml = "CART_example.xml")
# Test the CART model.
acc = test_CART(tree, dataset_torch)
print(acc)  

65.10012259910094


### Evaluation CART with depth 3.

In [ ]:
# Train a new tree for a specific dataset.
tree = train_CART(dataset_torch, max_CART_depth = 3, name_xml = "CART_example.xml")
# Test the CART model.
acc = test_CART(tree, dataset_torch)
print(acc)  

65.8152840212505


# Evaluation CART random partitions

### Evaluation with depth 2

In [ ]:
results = partition_validation(dataset_torch, 2, 10)
print("\nMean:\t" + str(statistics.mean(results)))
print("STD:\t" + str(statistics.stdev(results)))

Evaluación de la partion 1:
Acc: 63.444520081688225
Evaluación de la partion 2:
Acc: 64.0571817562968
Evaluación de la partion 3:
Acc: 64.32947583390062
Evaluación de la partion 4:
Acc: 63.30837304288631
Evaluación de la partion 5:
Acc: 63.98910823689585
Evaluación de la partion 6:
Acc: 63.58066712049013
Evaluación de la partion 7:
Acc: 64.60176991150442
Evaluación de la partion 8:
Acc: 66.3716814159292
Evaluación de la partion 9:
Acc: 65.14635806671205
Evaluación de la partion 10:
Acc: 65.48672566371681

Mean:	64.43158611300204
STD:	0.9845197708153681


### Evaluation with depth 3

In [ ]:
results = partition_validation(dataset_torch, 3, 10)
print("\nMean:\t" + str(statistics.mean(results)))
print("STD:\t" + str(statistics.stdev(results)))

Evaluación de la partion 1:
Acc: 64.39754935330157
Evaluación de la partion 2:
Acc: 63.30837304288631
Evaluación de la partion 3:
Acc: 65.28250510551395
Evaluación de la partion 4:
Acc: 64.80599046970728
Evaluación de la partion 5:
Acc: 64.60176991150442
Evaluación de la partion 6:
Acc: 64.73791695030633
Evaluación de la partion 7:
Acc: 61.81075561606535
Evaluación de la partion 8:
Acc: 64.32947583390062
Evaluación de la partion 9:
Acc: 65.14635806671205
Evaluación de la partion 10:
Acc: 63.78488767869299

Mean:	64.22055820285908
STD:	1.0334826767408063


# Implementation of Random Forest

In [ ]:
# Class to generate a random forest.
class RandomForest:
    def __init__(self, quantity_trees, depth):
        self.quantity_trees = quantity_trees
        self.depth = depth
        self.trees = []
    
    # Create K partition to train each of them with a CART.
    def train_random_forest(self, dataset_torch):
        # Generate k partitions from a dataset.
        kf = KFold(n_splits = self.quantity_trees)
        for train_index, test_index in kf.split(dataset_torch):
            test = dataset_torch[test_index]
            train = dataset_torch[train_index]

            # Train a new tree with the partition.
            tree = train_CART(train, max_CART_depth = self.depth, name_xml = "CART_example.xml")

            # Save the trees in a list.
            self.trees.append(tree)

            # Evaluate the created tree.
            acc = test_CART(tree, dataset_torch)

    # For a single observation calculate the estimator using vote scheme seen in classes.
    def evaluate_random_forest(self, input_torch):
        estimation = []
        for tree in self.trees:
            result = tree.evaluate_input(input_torch)
            estimation.append(result)
        
        # estimate = mode(estimation)
        estimate = max(set(estimation), key = estimation.count)
        return estimate
    
    # Calculate the random forest accuracy.
    def test_random_forest(self, testset_torch):
        sum  = 0
        for i in range(testset_torch.size()[0]):
            result = self.evaluate_random_forest(testset_torch[i])
            if testset_torch[i][4] == result:
                sum +=1
        accuracy = sum / testset_torch.size()[0] * 100;
        return accuracy
     

# Generating a new random forest.

* A new random forest with depth = 3 and quantity of CARTs = 5.


In [ ]:
random_forest = RandomForest(5, 3)

* Start training.

In [ ]:
random_forest.train_random_forest(dataset_torch)

* Evaluate random forest.

In [ ]:
random_forest.test_random_forest(dataset_torch)

65.12055578259093

# Random Forest Evaluation

* Fucntion to make 10 partition for test examples.

In [ ]:
def RF_Validation(dataset_torch, max_CART_depth, num_splits, num_CARTs):
    rs = ShuffleSplit(n_splits = num_splits, test_size=.30)
    cont_part = 1
    results = []
    # Getting index for N different datasets.
    for train_index, test_index in rs.split(dataset_torch):
        print("Evaluación de la partion %s:" % (cont_part))
        train = dataset_torch[train_index]
        test = dataset_torch[test_index]

        # Training a new forest.
        random_forest = RandomForest(num_CARTs, max_CART_depth)
        random_forest.train_random_forest(train)

        # Evaluation Accuracy.
        acc = random_forest.test_random_forest(test)
        print("Acc = %s" % (acc))
        cont_part += 1
        results.append(acc)
    return results

* Execute the fucntion.

In [ ]:
results = RF_Validation(dataset_torch=dataset_torch, max_CART_depth=3, num_splits=10, num_CARTs=5)
print("\nMean:\t" + str(statistics.mean(results)))
print("STD:\t" + str(statistics.stdev(results)))


Evaluación de la partion 1:
Acc = 64.19332879509871
Evaluación de la partion 2:
Acc = 66.98434309053778
Evaluación de la partion 3:
Acc = 63.852961198093936
Evaluación de la partion 4:
Acc = 64.94213750850919
Evaluación de la partion 5:
Acc = 66.3716814159292
Evaluación de la partion 6:
Acc = 65.82709326072158
Evaluación de la partion 7:
Acc = 66.30360789652825
Evaluación de la partion 8:
Acc = 65.69094622191967
Evaluación de la partion 9:
Acc = 64.60176991150442
Evaluación de la partion 10:
Acc = 63.78488767869299

Mean:	65.25527569775358
STD:	1.1369160895028267
